*   **Año:** [INSERTAR AÑO]
*   **Alumno/a:** [INSERTAR NOMBRE]
*   **Legajo:** [LEGAJO]

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [1]:
from pydantic import BaseModel


Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [4]:
class Client(BaseModel):
    dni: int
    nacionalidad: str
cliente = Client(dni=44254009, nacionalidad="Argentina")
print(cliente)

dni=44254009 nacionalidad='Argentina'


Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [6]:
class Client(BaseModel):
    dni: int
    nacionalidad: str

usuario = Client(dni=39755010, nacionalidad="Argentina")

print(f"DNI: {usuario.dni}")
print(f"Nacionalidad: {usuario.nacionalidad}")


DNI: 39755010
Nacionalidad: Argentina


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [2]:
from pydantic import BaseModel, StrictInt
class Client(BaseModel):
    dni: StrictInt
    nacionalidad: str
cliente_valido = Client(dni=39755010, nacionalidad="Argentina")
print("Usuario válido creado:", cliente_valido)

Usuario válido creado: dni=39755010 nacionalidad='Argentina'


Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [7]:
from pydantic import BaseModel, Field, ValidationError, validator
from datetime import date

NACIONALIDADES_VALIDAS = ["Argentina", "Brasil", "Chile", "Uruguay", "Paraguay"]

class Cliente(BaseModel):
    dni: int = Field(..., ge=0, description="El DNI debe ser un número positivo.")
    nacionalidad: str = Field(..., description="Debe ser una nacionalidad válida.")
    fecha_registro: date = Field(..., description="La fecha de registro no puede ser futura.")

    @validator("nacionalidad")
    def validar_nacionalidad(cls, value):
        if value not in NACIONALIDADES_VALIDAS:
            raise ValueError(f"Nacionalidad '{value}' no es válida. Debe ser una de {NACIONALIDADES_VALIDAS}.")
        return value

    @validator("fecha_registro")
    def validar_fecha_registro(cls, value):
        if value > date.today():
            raise ValueError("La fecha de registro no puede ser del futuro.")
        return value


<ipython-input-7-2b27bff1291e>:12: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator("nacionalidad")
<ipython-input-7-2b27bff1291e>:18: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator("fecha_registro")


In [8]:
try:
    cliente_valido = Cliente(dni=12345678, nacionalidad="Argentina", fecha_registro=date(2023, 11, 22))
    print("Cliente válido creado:", cliente_valido)
except ValidationError as e:
    print("Error en la validación:", e)


Cliente válido creado: dni=12345678 nacionalidad='Argentina' fecha_registro=datetime.date(2023, 11, 22)


In [9]:
try:
    cliente_invalido = Cliente(dni=12345678, nacionalidad="Marsiano", fecha_registro=date(2023, 11, 22))
except ValidationError as e:
    print("Error en la validación:", e)


Error en la validación: 1 validation error for Cliente
nacionalidad
  Value error, Nacionalidad 'Marsiano' no es válida. Debe ser una de ['Argentina', 'Brasil', 'Chile', 'Uruguay', 'Paraguay']. [type=value_error, input_value='Marsiano', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [10]:
try:
    cliente_invalido = Cliente(dni=-12345678, nacionalidad="Chile", fecha_registro=date(2023, 11, 22))
except ValidationError as e:
    print("Error en la validación:", e)


Error en la validación: 1 validation error for Cliente
dni
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-12345678, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than_equal


# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [23]:
import pandas as pd
import pandera as pa
from datetime import datetime

schema = pa.DataFrameSchema({
    "dni": pa.Column(int, checks=pa.Check.ge(0), nullable=False, description="El DNI debe ser un número positivo."),
    "nacionalidad": pa.Column(str, nullable=False, description="La nacionalidad debe ser válida."),
    "fecha_registro": pa.Column(
        pd.Timestamp,
        checks=pa.Check.le(pd.Timestamp(datetime.now())),
        nullable=False,
        description="La fecha de registro no puede ser del futuro."
    ),
})

data = {
    "dni": [12345678, 23456789, 34567890],
    "nacionalidad": ["Argentina", "Brasil", "Chile"],
    "fecha_registro": [datetime.now(), datetime(2025, 1, 1), datetime.now()],
}

df = pd.DataFrame(data)

print("DataFrame inicial:")
print(df)


current_time = datetime.now()


df["fecha_registro"] = df["fecha_registro"].apply(
    lambda x: x if x <= current_time else current_time
)

print("\nDataFrame corregido (fechas futuras ajustadas):")
print(df)

try:
    schema.validate(df)
    print("\nEl DataFrame es válido.")
except pa.errors.SchemaError as e:
    print("\nError en la validación:")
    print(e)


DataFrame inicial:
        dni nacionalidad             fecha_registro
0  12345678    Argentina 2024-11-22 23:07:09.054166
1  23456789       Brasil 2025-01-01 00:00:00.000000
2  34567890        Chile 2024-11-22 23:07:09.054170

DataFrame corregido (fechas futuras ajustadas):
        dni nacionalidad             fecha_registro
0  12345678    Argentina 2024-11-22 23:07:09.054166
1  23456789       Brasil 2024-11-22 23:07:09.058625
2  34567890        Chile 2024-11-22 23:07:09.054170

Error en la validación:
Column 'fecha_registro' failed element-wise validator number 0: less_than_or_equal_to(2024-11-22 23:07:09.049193) failure cases: 2024-11-22 23:07:09.054166, 2024-11-22 23:07:09.058625, 2024-11-22 23:07:09.054170
